# Weather Health Risk Prediction Model

This notebook trains and evaluates models for predicting health risks from weather data.

## Objectives:
1. Load weather data
2. Derive health risk levels from weather conditions
3. Train multiple model types
4. Evaluate and compare models
5. Save best model


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings

sys.path.append(str(Path().absolute().parent / "src"))
from models.weather_model import WeatherHealthRiskModel

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

print("Libraries imported successfully!")


Libraries imported successfully!


In [2]:
# Load data
data_dir = Path().absolute().parent / "data" / "raw"
weather_file = data_dir / "weather_data.csv"

df = pd.read_csv(weather_file, parse_dates=['timestamp'])
print(f"✓ Loaded {len(df)} records")
print(f"Dataset shape: {df.shape}")


✓ Loaded 150 records
Dataset shape: (150, 10)


In [3]:
# Train multiple model types
model_types = ['random_forest', 'gradient_boosting', 'logistic_regression']
results = {}

for model_type in model_types:
    print(f"\n{'='*60}")
    print(f"Training {model_type.upper()}")
    print(f"{'='*60}")
    
    model = WeatherHealthRiskModel(model_type=model_type)
    # Model will derive health_risk_level from weather conditions
    metrics = model.train(df, target_col=None, test_size=0.2)
    
    results[model_type] = {
        'model': model,
        'metrics': metrics
    }



Training RANDOM_FOREST
Deriving health risk levels from weather conditions...
Training random_forest model...

Model Performance:
Accuracy: 0.9333
Precision: 0.9422
Recall: 0.9333
F1-Score: 0.9331
ROC-AUC: 1.0000

Classification Report:
              precision    recall  f1-score   support

        high       1.00      0.80      0.89         5
         low       1.00      0.92      0.96        12
    moderate       0.87      1.00      0.93        13

    accuracy                           0.93        30
   macro avg       0.96      0.91      0.92        30
weighted avg       0.94      0.93      0.93        30


Training GRADIENT_BOOSTING
Deriving health risk levels from weather conditions...
Training gradient_boosting model...

Model Performance:
Accuracy: 0.9667
Precision: 0.9690
Recall: 0.9667
F1-Score: 0.9666
ROC-AUC: 0.9660

Classification Report:
              precision    recall  f1-score   support

        high       1.00      1.00      1.00         5
         low       1.00   

In [4]:
# Compare models and save best
comparison_df = pd.DataFrame({
    model_type: {
        'Accuracy': results[model_type]['metrics']['accuracy'],
        'Precision': results[model_type]['metrics']['precision'],
        'Recall': results[model_type]['metrics']['recall'],
        'F1-Score': results[model_type]['metrics']['f1_score'],
        'ROC-AUC': results[model_type]['metrics'].get('roc_auc', None)
    }
    for model_type in model_types
})

print("Model Comparison:")
print(comparison_df.round(4))

# Select best model
best_model_type = max(model_types, key=lambda x: results[x]['metrics']['f1_score'])
best_model = results[best_model_type]['model']

models_dir = Path().absolute().parent / "models"
models_dir.mkdir(exist_ok=True)
model_path = models_dir / f"weather_model_{best_model_type}.pkl"
best_model.save(str(model_path))
print(f"\n✓ Best model ({best_model_type}) saved to: {model_path}")


Model Comparison:
           random_forest  gradient_boosting  logistic_regression
Accuracy          0.9333             0.9667               0.8333
Precision         0.9422             0.9690               0.8583
Recall            0.9333             0.9667               0.8333
F1-Score          0.9331             0.9666               0.8205
ROC-AUC           1.0000             0.9660               0.9614
Model saved to /Users/faiqahmed/Desktop/Semesters/Semester7/MLOPS/PROJECT/models/weather_model_gradient_boosting.pkl

✓ Best model (gradient_boosting) saved to: /Users/faiqahmed/Desktop/Semesters/Semester7/MLOPS/PROJECT/models/weather_model_gradient_boosting.pkl
